In [8]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# Load data set

In [31]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [42]:
trainset = torchvision.datasets.CIFAR10(root='./cifardata', train=True,
                                       download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=8, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./cifardata', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=8, shuffle=True, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck') # 클래스가 10개다.

Files already downloaded and verified
Files already downloaded and verified


# Build a model

In [33]:
import torch.nn as nn
import torch.nn.functional as F

In [34]:
# 파이토치에서 모델을 만드는 가장 기본적인 방법은 다음과 같다.

class kenGwonNeuralNet(nn.Module): # 여기서 반드시 nn.Module을 상속 받아야 한다.
    
    def __init__(self): ## __init__()에서 내가 만들고자 하는 뉴럴넷의 구조를 적어주면 된다. pretrained model이 여기 들어와야 하는 것 같다.
        super(kenGwonNeuralNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5* 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10) # 분류하기 위한 클래스가 10개이기 때문에 아웃풋이 10개인 레이어를 이렇게 만들었다. 

    def forward(self, x): ## forward()에서 연산의 순서를 정의해준다. __init__()에서 만들어놓은 레이어들을 데이터 x에 대해서 어떻게 연산을 시킬 것인지를 적는 것이다.
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16*5*5) # 위에서 convolution레이어를 다 돌았으니 이제 flatten해서 선형 레이어에 넣어주기 위해서 torch.Tensor를 변환해주었다.
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


myNN = kenGwonNeuralNet()

In [35]:
print(myNN)

kenGwonNeuralNet(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


# Implement the model with trainig set

In [36]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss() # Loss function로 전형적인 분류문제를 위해 쓰이는 CrossEntropyLoss()를 사용하기로 함 
optimizer = optim.SGD(myNN.parameters(), lr=0.001, momentum=0.9) # myNN.parameters() 이것을 통해 우리가 선언한 뉴럴넷의 구조가 옵티마이저에 들어가게 된다.

In [38]:
# 이 블럭이 실질적으로 학습이 일어나는 부분이다.

for epoch in range(3):
    
    running_loss = 0.0
    
    for i, data in enumerate(trainloader, 0):
        images, labels = data # batch 묶음으로 던져주기 때문에, 변수명을 복수형으로 써주는게 맞다. 위에서 batch를 8로 줬기 때문에, 이 한번의 for문에서 연산되는 이미지의 갯수는 8개이다.

        optimizer.zero_grad() # 옵티마이저를 초기화 시켜주는 작업 (zero the parameter gradients)

        # forward + backward + optimize
        outputs = myNN(images)
        loss = criterion(outputs, labels) # 뉴럴넷을 통과해서 나온 값을 실제 레이블과 비교하여 loss를 계산해야 함. 여기서는 8개 이미지에 대한 하나의 LOSS값을 얻게됨
        loss.backward() # 위해서 구해진 loss값으로 weight를 수정하기 위해 backpropagation을 함
        optimizer.step() # 옵티마이저의 설정값에 따라 그에 상응하는 크기로 weight를 수정함

        # print statistics
        running_loss += loss.item() # loss는 scalar형의 단일 숫자값으로 나오기 때문에 거기서 값만 딱 빼내기 위해서 item()함수를 체이닝 했다.
        if i % 200 == 199: # batch 2000번 돌때마다 loss값이 어떻게 변하고 있는지 출력하겠다는 뜻
            print(f"[epoch: {epoch + 1}, batch: {i + 1:5d} loss: {running_loss / 2000:.3f}]")
            running_loss = 0.0


print("학습 완료")        

[epoch: 1, batch:   200 loss: 0.164]
[epoch: 1, batch:   400 loss: 0.162]
[epoch: 1, batch:   600 loss: 0.163]
[epoch: 1, batch:   800 loss: 0.160]
[epoch: 1, batch:  1000 loss: 0.156]
[epoch: 1, batch:  1200 loss: 0.158]
[epoch: 2, batch:   200 loss: 0.154]
[epoch: 2, batch:   400 loss: 0.146]
[epoch: 2, batch:   600 loss: 0.151]
[epoch: 2, batch:   800 loss: 0.150]
[epoch: 2, batch:  1000 loss: 0.151]
[epoch: 2, batch:  1200 loss: 0.150]
[epoch: 3, batch:   200 loss: 0.144]
[epoch: 3, batch:   400 loss: 0.142]
[epoch: 3, batch:   600 loss: 0.142]
[epoch: 3, batch:   800 loss: 0.143]
[epoch: 3, batch:  1000 loss: 0.141]
[epoch: 3, batch:  1200 loss: 0.143]
학습 완료


# Save the trained model

In [39]:
PATH = './cifar_net.pth'
torch.save(myNN.state_dict(), PATH)

모델을 저장하는 것은 반드시 모든 epoch이 끝나고 할 필요는 없다. 무조건 많은 epoch을 돈다고 해서 좋은 모델이 나오는 것이 아니다.
학습을 하는 도중에 best model이 나왔을 수도 있다. 바로 그 순간 model을 저장하도록 구성할 수 있다. 
그렇게 하려면 이렇게 모델을 save하는 코드를 위에서 모델을 trainning하는 코드 중간에 넣으면 된다.

# Load the pre-trained model

In [40]:
ptNN = kenGwonNeuralNet()
ptNN.load_state_dict(torch.load(PATH))

<All keys matched successfully>

#### output은 mini batch의 결과가 산출되기 때문에 for문을 통해서 test 전체의 예측값을 구해야 한다.

In [46]:
correct = 0
total = 0

with torch.no_grad(): # 실제로 test data에 대해서 예측을 할 때는 gradient를 업데이트 하면 안되기 때문에 반드시 이 구문으로 감싸주고 테스트 데이터를 돌려봐야 한다.
    for data in testloader:
        images, labels = data
        outputs = ptNN(images) # 우리가 만든 뉴럴넷의 최종 레이어는 output으로 10개의 값을 리턴하도록 되어있다.
        _, predicted = torch.max(outputs.data, 1) # 그 10개의 값들 중에서 가장 값이 큰 것이 그 클래스일 확률이 높다는 것으로 보는 것이다. np.argmax같은 것이다.
        
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'1000개의 테스트 이미지에 대한 정확도: {100 * correct / total} %')

1000개의 테스트 이미지에 대한 정확도: 52.79 %
